# Project Name:- Identification of User Query on Stack Overflow Using Semantic Search
### Semantic search seeks to improve search accuracy by understanding the searcher's intent.

## Objectives of Project:-

### 1.To identify most relevant questions to a query [text similarity]
### 2.Identify the matching tags and pick top relevant questions from stack overflow.
### 3.To identify top k solutions of the problem. (sentiment analysis of review content)

In [2]:
import json
import requests
import ibm_watson
Auth_key = "rEi)4Xt7RnJf4tqP)pk5nQ(("
from html import unescape

## Removal of Stopwords like I, am , the , etc

In [3]:
def remove_stopwords(words_list_eta):
    from nltk.corpus import stopwords
    import nltk
    # nltk.download('stopwords')

    stop_words = stopwords.words('english')

    return [word for word in words_list_eta if word not in stop_words]

## Check if it exists in stackoverflow tags

In [4]:
with open('SO_tags_dict_2.txt', 'r') as f:
    value_list = json.load(f)
f.close()

def remove_nonsotag(words_list_eta):
    for i in words_list_eta:
        if i not in value_list:
            words_list_eta.remove(i)
            
    local_list_eta = set(words_list_eta)
    return local_list_eta

## Watson Assistant

In [5]:
credentials_moh = {"api_key": "HTHsjdKJzDGeQlHu_uJijwKbb48__vZbypti5MVchZL7",
                   "api_url": "https://gateway.watsonplatform.net/assistant/api",
               "workspace_id":"f2d82aa9-b6a3-4657-a6eb-01ff1cfec5a8"
              }

In [76]:
def watson_assistantv1(query_eta):
    assistant2 = ibm_watson.AssistantV1(
        version='2019-02-28',
        url=credentials_moh['api_url'],
        iam_apikey=credentials_moh['api_key']
    )    

    response2 = assistant2.message(
        workspace_id=credentials_moh['workspace_id'],
        input={
            'text': query_eta
        }
    ).get_result()  
            
#     query2 = []
#     tags2 = {}
    tags2 = []
    for i in response2['entities']:
#         if i['entity'] == 'query':
#             query2.append(i['value'])

        if i['entity'] == 'Stack_Overflow_Tag':
            if i['confidence'] > 0.8:
                tags2.append(i['value'])
#             tags2.append(i['value'] + ':' +str(i['confidence']))

    if len(tags2) >= 1:
        return(tags2)

## Watson NLU

In [44]:
def watson_nlu(text_test_eta):
    import json
    from ibm_watson import NaturalLanguageUnderstandingV1
    from ibm_watson.natural_language_understanding_v1 import Features,KeywordsOptions ,SyntaxOptions, SyntaxOptionsTokens,SentimentOptions,EntitiesOptions

    credentials = {"api_key": "wEHseKE5Iqv2PfZwtOP6FVQYOeoJxIa-32Xu5QbNkBRM",
                   "api_url": "https://gateway.watsonplatform.net/natural-language-understanding/api",
                   "model":"9ccb4de2-2bf1-410a-8d51-36f77f9cf907"
                  }


    natural_language_understanding = NaturalLanguageUnderstandingV1(
        version='2018-11-16',
        iam_apikey=credentials['api_key'],
        url=credentials['api_url']
    )
    response_eta = natural_language_understanding.analyze(
        text=text_test_eta,
        features=Features(entities=EntitiesOptions(limit=1,model=credentials['model']))).get_result()
#     print(response_eta['usage'])
    return response_eta

In [83]:
def most_relevant_tags(query_eta):
    assistant_list = set(watson_assistantv1(query_eta))
    local_list = set(remove_nonsotag(remove_stopwords(query_eta.split())))
    final_list_eta = assistant_list.union(local_list)
#     print("local_list is ",local_list,"assistant_list is",assistant_list)
    return final_list_eta

In [89]:
def remove_error_tags(query_eta,final_list_eta):
    response = watson_nlu(query)
    error_tag = []
    for tag in response['entities']:
        try:
            if tag['type']== 'NO_SO_TAG':
                deviated_version = watson_assistantv1(tag['text'])
                print("Removing",deviated_version," from tags as it is not in context")
                for item in deviated_version:
                    error_tag.append(item)
        except Exception as e:
            print(e)
    final_list_changed = set(final_list_eta).difference(error_tag)
    return list(final_list_changed)

## Let's get top 30 questions with are relevant to the tags and also title.

In [90]:
query = "where is python path on windows"

In [92]:
final_list = most_relevant_tags(query)
print(final_list)

{'windows', 'where-clause', 'python', 'path'}


In [93]:
final_list = remove_error_tags(query,final_list)

Removing ['where-clause']  from tags as it is not in context


['windows', 'python', 'path']

In [10]:
api_url_1 = "https://api.stackexchange.com/2.2/similar?key="+Auth_key+"&order=desc&sort=relevance&tagged="+";".join(final_list)+"&title="
api_url_2 = query+"&site=stackoverflow"
api_url = api_url_1 + api_url_2
api_url

'https://api.stackexchange.com/2.2/similar?key=rEi)4Xt7RnJf4tqP)pk5nQ((&order=desc&sort=relevance&tagged=app;node.js&title=how to run app node.js&site=stackoverflow'

In [11]:
# let's get top 30 questions
import requests
top_relevant_questions = requests.get(api_url)
top_relevant_questions = top_relevant_questions.json()
print(len(top_relevant_questions['items']))
#Please don't add other code in this cell because it will use api again and again while testing, Mohit

30


## Now let's make a dataframe of the above questions and analyse them

In [12]:
from pandas import DataFrame
df = DataFrame(top_relevant_questions['items'])

In [13]:
df

,accepted_answer_id,answer_count,closed_date,closed_reason,creation_date,is_answered,last_activity_date,last_edit_date,link,owner,question_id,score,tags,title,view_count
0,42246960.0,1,NaN,NaN,1487141626,True,1487251207,1.487240e+09,https://stackoverflow.com/questions/42242498/h...,"{'reputation': 276, 'user_id': 7104874, 'user_...",42242498,3,"[javascript, node.js, express, gruntjs]",How to run node.js app by grunt?,1763
1,NaN,0,NaN,NaN,1562426956,False,1562426956,NaN,https://stackoverflow.com/questions/56915448/h...,"{'reputation': 13, 'user_id': 11317272, 'user_...",56915448,0,"[node.js, apache, passenger, namecheap]",How to run Node.js app on Phusion Passenger app?,7
2,NaN,0,NaN,NaN,1430289347,False,1430289347,NaN,https://stackoverflow.com/questions/29936030/h...,"{'reputation': 34, 'user_id': 2362113, 'user_t...",29936030,1,"[android, node.js, npm, adb]",How to run Node.js app in Android Mobile,222
3,NaN,2,1.543594e+09,duplicate,1543562416,True,1543569889,1.543570e+09,https://stackoverflow.com/questions/53552966/h...,"{'reputation': 1, 'user_id': 10416346, 'user_t...",53552966,-1,"[javascript, node.js, localhost]",How to run Node.js in localhost?,3583
4,6455134.0,10,NaN,NaN,1303991814,True,1545037206,NaN,https://stackoverflow.com/questions/5818202/ho...,"{'reputation': 5479, 'user_id': 591939, 'user_...",5818202,50,[node.js],How to run node.js app forever when console is...,52100
5,32277213.0,3,NaN,NaN,1440562603,True,1497351166,NaN,https://stackoverflow.com/questions/32217907/h...,"{'reputation': 116, 'user_id': 3494471, 'user_...",32217907,1,"[node.js, azure, azure-web-sites, azure-webjobs]",How to run node.js in azure webjobs?,4361
6,37601577.0,2,NaN,NaN,1464621727,True,1504880838,1.467323e+09,https://stackoverflow.com/questions/37529156/h...,"{'reputation': 3537, 'user_id': 972240, 'user_...",37529156,2,"[javascript, node.js, ecmascript-6, koa, koa2]",How to run simple app with koa2?,1356
7,NaN,0,NaN,NaN,1412504728,False,1412504728,NaN,https://stackoverflow.com/questions/26201710/n...,"{'reputation': 15, 'user_id': 3472044, 'user_t...",26201710,0,"[linux, node.js]",node.js how to run mount command,182
8,23184506.0,1,NaN,NaN,1397967314,True,1398038322,NaN,https://stackoverflow.com/questions/23178370/h...,"{'reputation': 19, 'user_id': 1181132, 'user_t...",23178370,0,"[node.js, angularjs, amazon-ec2, gruntjs, yeoman]",How to run a node.js app on ec2,1333
9,NaN,0,NaN,NaN,1376056483,False,1376056483,NaN,https://stackoverflow.com/questions/18148344/h...,"{'reputation': 551, 'user_id': 2131176, 'user_...",18148344,0,"[asp.net, node.js, iisnode]",how to run node app on iisnode in Visual Studio,794


## 1st Phase(No statistics considered)

### 1.) Separate rows with closed_reason as duplicate, to find the question of which it is duplicate, find them and      merge those question in the df after 1st phase if they match a certain criteria.
### 2.) Remove rows with closed_reason
### 3.)Remove questions with 0 answers.(if is_answered == False)
### 4.) What to do with Negative Scores/Downvotes?

In [14]:
# Getting df_duplicate to be handled later.
try:
    df_duplicate = df[df.closed_reason == 'duplicate']
    print(len(df_duplicate),"Closed questions because of duplicacy")
except Exception as e: print(e)

2 Closed questions because of duplicacy


In [15]:
# Removal of closed questions
try:
    df = df[df.closed_reason != 'duplicate']
    df = df[df.closed_reason != 'off-topic']
    df = df[df.closed_reason != "unclear what you're asking"]
    df = df[df.closed_reason != "too broad"]
    df = df[df.closed_reason != "primarily opinion-based"]
except Exception as e: print(e)

In [16]:
# Removal of question with 0 ansswers
df = df[df.is_answered == True]
len(df)

23

In [17]:
# Removal of closed_reason and closed_date columns and others as they are unnecessary now
try:
    del df['closed_reason']
    del df['closed_date']
    del df['creation_date']
    del df['is_answered']
    del df['last_activity_date']
    del df['last_edit_date']
except Exception as e: print(e)
len(df)

In [42]:
df = df[df.score>0]
len(df)

0

## 2nd Phase(based on some statistics)
### 1.) Take top 25% questions on the basis of following:-
#### a.) No of  StackOverflow tags of question matching with query tags by NLU/watson assistant
#### b.) Text Similarity of the question title with query. (using tf-idf(nltk),intent classification,watson NLC,etc)
#### c.)  Keyword using nlu for query and title

#### d.) Sentiment Analysis of the comments to check if the question is nicely edited or not. If Positive comments than the question is nice but if negative comments than that question is not nice. (Keeping in mind that only appropriate comments can be there on stackoverflow as moderators remove unecessary comments?)


## a.) No of  StackOverflow tags of question matching with query tags

In [19]:
df['Tag_Match'] = df.apply(lambda row: len(set(row.tags).intersection(final_list)), axis = 1)

In [20]:
print(query)

how to run app node.js


In [21]:
# df.sort_values(by = ["Tag_Match"],ascending = False)

# b.) Text Similarity of the question title with query. (using tf-idf(nltk),intent classification,watson NLC,etc)

### Try with TF-IDF NLTK

# NEEDS IMPROVEMENT (changed in the final implementation)

In [22]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

# nltk.download('punkt')

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
#     print((tfidf * tfidf.T).A)
    return ((tfidf * tfidf.T).A)[0,1]
    
print (cosine_sim(df.iloc[0].title, query))
print (cosine_sim(df.iloc[1].title, query))
print (cosine_sim(df.iloc[3].title, query))

0.7765145304745156
0.5797386715376658
0.4112070550676187


In [23]:
df['Similarity_index'] = df.apply(lambda row:cosine_sim(row.title,query) , axis = 1)

In [24]:
print(query)

how to run app node.js


In [25]:
# df.sort_values(by = ["Similarity_index"],ascending = False)

# c) Keywords of title and query

## Applying NLU to get keywords of every question title

In [26]:
#watson

df['keyword_in_title'] = df.apply(lambda row:most_relevant_tags(row.title) , axis = 1)

In [27]:
df['Keywords_Match'] = df.apply(lambda row: len(set(row.keyword_in_title).intersection(final_list)), axis = 1)

In [28]:
# del df['keyword_in_title']

In [29]:
# df.sort_values(by = ["Keywords_Match"],ascending = False)

# Let's apply Statistical and other measure on the newly created columns created above.

### Parameters

In [30]:
alpha = 0.5
beta = 1.5
gamma = 1.2

In [31]:
df['Final_function'] = df.apply(lambda row:(alpha*row.Tag_Match)+(beta*row.Similarity_index)+(gamma*row.Keywords_Match),axis = 1)

In [32]:
df = df.sort_values(by = ["Final_function"],ascending = False)

In [33]:
print(query)

how to run app node.js


In [34]:
df

,accepted_answer_id,answer_count,link,owner,question_id,score,tags,title,view_count,Tag_Match,Similarity_index,keyword_in_title,Keywords_Match,Final_function
25,NaN,2,https://stackoverflow.com/questions/44591120/i...,"{'reputation': 1234, 'user_id': 311263, 'user_...",44591120,1,"[node.js, heroku]",Is there a way to run non web Node.js app on H...,441,1,0.524591,"{app, web, heroku, non, way, node.js}",2,3.686887
0,42246960.0,1,https://stackoverflow.com/questions/42242498/h...,"{'reputation': 276, 'user_id': 7104874, 'user_...",42242498,3,"[javascript, node.js, express, gruntjs]",How to run node.js app by grunt?,1763,1,0.776515,"{run, grunt?, gruntjs, node.js}",1,2.864772
8,23184506.0,1,https://stackoverflow.com/questions/23178370/h...,"{'reputation': 19, 'user_id': 1181132, 'user_t...",23178370,0,"[node.js, angularjs, amazon-ec2, gruntjs, yeoman]",How to run a node.js app on ec2,1333,1,0.776515,"{amazon-ec2, run, ec2, node.js}",1,2.864772
22,22612532.0,2,https://stackoverflow.com/questions/22612052/w...,"{'reputation': 741, 'user_id': 1683942, 'user_...",22612052,-2,[node.js],Where to run node.js,166,1,0.709297,"{run, where-clause, node.js}",1,2.763946
19,33851715.0,1,https://stackoverflow.com/questions/33850777/h...,"{'reputation': 3453, 'user_id': 2517730, 'user...",33850777,0,"[node.js, azure, heroku]",How to run a node.js app in background on Azure,495,1,0.656973,"{background, run, azure, node.js}",1,2.685459
4,6455134.0,10,https://stackoverflow.com/questions/5818202/ho...,"{'reputation': 5479, 'user_id': 591939, 'user_...",5818202,50,[node.js],How to run node.js app forever when console is...,52100,1,0.579739,"{run, forever, console, node.js}",1,2.569608
27,34580053.0,1,https://stackoverflow.com/questions/34579550/h...,"{'reputation': 62, 'user_id': 3548251, 'user_t...",34579550,3,"[node.js, mongodb, cordova, ionic-framework]",How to run Node.js server in Ionic mobile app?,7095,1,0.579739,"{run, mobile, ionic-framework, server, node.js}",1,2.569608
13,NaN,1,https://stackoverflow.com/questions/32635259/u...,"{'reputation': 2627, 'user_id': 841915, 'user_...",32635259,4,"[javascript, node.js, appjs]",Unable to run appjs sample app using node.js,1944,1,0.524591,"{run, sample, appjs, using, node.js}",1,2.486887
16,32512298.0,1,https://stackoverflow.com/questions/32512193/n...,"{'reputation': 53, 'user_id': 3167167, 'user_t...",32512193,-1,[node.js],node.js i have the program but how to run,88,1,0.503103,"{run, node.js}",1,2.454654
24,32468160.0,1,https://stackoverflow.com/questions/32430224/h...,"{'reputation': 38, 'user_id': 2643948, 'user_t...",32430224,0,"[node.js, selenium, webdriver-io]",How to run Webdriver.io heedlessly in node.js,143,1,0.411207,"{run, heedlessly, webdriver-io, node.js}",1,2.316811


# RESULTS TOP 5

In [35]:
question_dict = {}
for i in range(5):
    print(df.iloc[i].title)
    print(df.iloc[i].link)
    question_dict[df.iloc[i].question_id]  = df.iloc[i].title

Is there a way to run non web Node.js app on Heroku?
https://stackoverflow.com/questions/44591120/is-there-a-way-to-run-non-web-node-js-app-on-heroku
How to run node.js app by grunt?
https://stackoverflow.com/questions/42242498/how-to-run-node-js-app-by-grunt
How to run a node.js app on ec2
https://stackoverflow.com/questions/23178370/how-to-run-a-node-js-app-on-ec2
Where to run node.js
https://stackoverflow.com/questions/22612052/where-to-run-node-js
How to run a node.js app in background on Azure
https://stackoverflow.com/questions/33850777/how-to-run-a-node-js-app-in-background-on-azure


In [36]:
question_dict

{22612052: 'Where to run node.js',
 23178370: 'How to run a node.js app on ec2',
 33850777: 'How to run a node.js app in background on Azure',
 42242498: 'How to run node.js app by grunt?',
 44591120: 'Is there a way to run non web Node.js app on Heroku?'}

# WORK ON ANSWERS GIVEN BY TOP 5 QUESTIONS

## 1.Accepted Answer[1st link if there is any]
## 2.Reputed Answer(Highest Upvoted and Owner Reputation)[all the links are ordered like this]

In [37]:
def badge_reputation(dict_eta):
    if "badge_counts" in dict_eta:
        alpha = 0.3
        beta = 0.5
        gamma = 0.9
        badge_repu = 0
        dict_eta_2 = dict_eta['badge_counts']
        if 'bronze' in dict_eta_2:
            badge_repu+=alpha*dict_eta_2['bronze']
        if 'silver' in dict_eta_2:
            badge_repu+=beta*dict_eta_2['silver']
        if 'gold' in dict_eta_2:
            badge_repu+=gamma*dict_eta_2['gold']
        return badge_repu
    else:
        return 0

In [38]:
def top_answers_fun(q_id_eta):
    # parameters
    delta = 0.5
    epsilon = 0.5
    zeta = 0.5
    url_eta = "https://api.stackexchange.com/2.2/questions/"+q_id_eta+"/answers?key="+Auth_key+"&order=desc&sort=votes&site=stackoverflow&filter=!)Q7ohwN5q2ex6NuB-d**q(*C"
    top_answers = requests.get(url_eta)
    top_answers = top_answers.json()
    df_ans = DataFrame(top_answers['items'])
#     print(df_ans)
    df_ans['badge_reputation'] = df_ans.apply(lambda row:badge_reputation(row.owner) , axis = 1)
    df_ans['repu'] = df_ans.apply(lambda row:row.owner['reputation'],axis=1)
    df_ans['final_function'] = df_ans.apply(lambda row:delta*row.badge_reputation+epsilon*row.repu+zeta*row.score,axis=1)
    df_ans = df_ans.sort_values(by = ["final_function"],ascending = False)
    eta = df_ans.iloc[0]['final_function']
    df_ans['final_function_2'] = df_ans.apply(lambda row:row.final_function+eta*int(row.is_accepted),axis=1) 
    del df_ans['owner']
    type_a = list(df_ans['share_link'])
    return type_a

In [43]:
for q_id in question_dict:
#     print(question_dict[q_id])
    for ans in top_answers_fun(str(q_id)):
        print(ans)

https://stackoverflow.com/a/44591947
https://stackoverflow.com/a/44591212
https://stackoverflow.com/a/42246960
https://stackoverflow.com/a/23184506
https://stackoverflow.com/a/22612532
https://stackoverflow.com/a/22612126
https://stackoverflow.com/a/33851715
